In [ ]:
# default_exp linear

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

# linear

> Linear functions helpers

In [ ]:
# export 
import numpy as np
from seal import Ciphertext, Plaintext, CKKSEncoder, DoubleVector, Evaluator, GaloisKeys
from typing import List

In [ ]:
# export
def pad_along_axis(array: np.ndarray, target_length, axis=0):
    """Pads an array to a given target length, on a given axis."""
    pad_size = target_length - array.shape[axis]
    axis_nb = len(array.shape)

    if pad_size <= 0:
        return array

    npad = [(0, 0) for x in range(axis_nb)]
    npad[axis] = (0, pad_size)

    b = np.pad(array, pad_width=npad, mode='constant', constant_values=0)

    return b

In [ ]:
# export
def arrays_to_ptx(arrays: List[np.ndarray], encoder: CKKSEncoder, scale: float) -> List[Plaintext]:
    """Converts an array of arrays to an array of plaintexts."""
    outputs = []
    for array in arrays:
        ptx = Plaintext()
        encoder.encode(DoubleVector(list(array)), scale, ptx)
        outputs.append(ptx)
    return outputs

In [ ]:
# export
def extract_diagonals(matrix: np.ndarray) -> List[np.ndarray]:
    """Extracts the diagonals of the matrix"""
    assert matrix.shape[0] == matrix.shape[1], "Non square matrix"
    dim = matrix.shape[0]
    
    diagonals = []
    
    for i in range(dim):
        diagonal = []
        for j in range(dim):
            diagonal.append(matrix[j][(j+i) % dim])
        diagonal = np.array(diagonal)
        diagonals.append(diagonal)
    return diagonals

In [ ]:
# export
def matrix_multiply_diagonals(diagonals: List[Plaintext], ctx: Ciphertext, 
                              evaluator: Evaluator, galois_keys: GaloisKeys):
    output = Ciphertext()
    
    for i in range(len(diagonals)):
        
        temp = Ciphertext()
        diagonal = diagonals[i]
        
        evaluator.rotate_vector(ctx, i, galois_keys, temp)
            
        evaluator.mod_switch_to_inplace(diagonal, temp.parms_id())
        evaluator.multiply_plain_inplace(temp, diagonal)
        evaluator.rescale_to_next_inplace(temp)
        
        if i == 0:
            output = temp
        else:
            evaluator.add_inplace(output, temp)

    return output

In [ ]:
# export
def sum_reduce(ctx, evaluator, galois_keys, n_slot):
    """Sums all the coefficients of the ciphertext, supposing that coefficients up to n_slot 
    are non zero. The first coefficient of the output will then be the sum of the coefficients."""
    n = int(np.ceil(np.log2(n_slot)))
    
    temp = Ciphertext()
    output = Ciphertext()
    
    for i in range(n):
        if i == 0:
            evaluator.rotate_vector(ctx, 2**i, galois_keys, temp)
            evaluator.add(ctx, temp, output)
        else:
            evaluator.rotate_vector(output, 2**i, galois_keys, temp)
            evaluator.add_inplace(output, temp)
    return output

def dot_product_plain(ctx, ptx, evaluator, galois_keys, n_slot):
    """Computes the dot product between a ciphertext and a plaintext"""
    output = Ciphertext()
    
    evaluator.multiply_plain(ctx, ptx, output)
    output = sum_reduce(output, evaluator, galois_keys, n_slot)
    
    return output

In [ ]:
import builtins
from cryptotree.seal_helper import print_ctx, print_ptx, create_seal_globals, append_globals_to_builtins
from seal import Plaintext, Ciphertext

poly_modulus_degree = 8192
moduli = [40,30,30,40]
PRECISION_BITS = 30

create_seal_globals(globals(), poly_modulus_degree, moduli, PRECISION_BITS)
append_globals_to_builtins(globals(), builtins)

In [ ]:
x = [1,2,3,4]
n_slot = len(x)

ptx = Plaintext()
encoder.encode(DoubleVector(x), scale, ptx)

ctx = Ciphertext()
encryptor.encrypt(ptx, ctx)

In [ ]:
print_ctx(sum_reduce(ctx, evaluator, galois_keys, n_slot))
print_ctx(dot_product_plain(ctx, ptx, evaluator, galois_keys, n_slot))


    [ 10.0000779, 9.0000166, 7.0000079, ..., 0.9999931, 3.0000874, 6.0000372 ]


    [ 29.9999957, 28.9999968, 24.9999951, ..., 0.9999989, 5.0000006, 13.9999975 ]

